In [1]:
from __future__ import print_function
import paddle
import paddle.fluid as fluid
import numpy
import sys

In [2]:
def vgg_bn_drop(input):
    def conv_block(ipt, num_filter, groups, dropouts):
        return fluid.nets.img_conv_group(
            input=ipt,
            pool_size=2,
            pool_stride=2,
            conv_num_filter=[num_filter] * groups,
            conv_filter_size=3,
            conv_act='relu',
            conv_with_batchnorm=True,
            conv_batchnorm_drop_rate=dropouts,
            pool_type='max')

    conv1 = conv_block(input, 64, 2, [0.3, 0])
    conv2 = conv_block(conv1, 128, 2, [0.4, 0])
    conv3 = conv_block(conv2, 256, 3, [0.4, 0.4, 0])
    conv4 = conv_block(conv3, 512, 3, [0.4, 0.4, 0])
    conv5 = conv_block(conv4, 512, 3, [0.4, 0.4, 0])

    drop = fluid.layers.dropout(x=conv5, dropout_prob=0.5)
    fc1 = fluid.layers.fc(input=drop, size=512, act=None)
    bn = fluid.layers.batch_norm(input=fc1, act='relu')
    drop2 = fluid.layers.dropout(x=bn, dropout_prob=0.5)
    fc2 = fluid.layers.fc(input=drop2, size=512, act=None)
    predict = fluid.layers.fc(input=fc2, size=10, act='softmax')
    return predict

In [3]:
def inference_program():
    # The image is 32 * 32 with RGB representation.
    data_shape = [3, 32, 32]
    images = fluid.layers.data(name='pixel', shape=data_shape, dtype='float32')

    predict = vgg_bn_drop(images)
    # predict = vgg_bn_drop(images) # un-comment to use vgg net
    return predict

In [4]:
def train_program():
    predict = inference_program()

    label = fluid.layers.data(name='label', shape=[1], dtype='int64')
    cost = fluid.layers.cross_entropy(input=predict, label=label)
    avg_cost = fluid.layers.mean(cost)
    accuracy = fluid.layers.accuracy(input=predict, label=label)
    return [avg_cost, accuracy]

In [5]:
def optimizer_program():
    return fluid.optimizer.Adam(learning_rate=0.0001)

In [6]:
# Each batch will yield 32 images
BATCH_SIZE = 64

# Reader for training
# image pixels in [0, 1] and label in [0, 99].
train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.cifar.train10(), buf_size=50000),
    batch_size=BATCH_SIZE)

# Reader for testing. A separated data set for testing.
test_reader = paddle.batch(
    paddle.dataset.cifar.test10(), batch_size=BATCH_SIZE)

In [7]:
use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

feed_order = ['pixel', 'label']

main_program = fluid.default_main_program()
star_program = fluid.default_startup_program()

avg_cost, acc = train_program()

# Test program
test_program = main_program.clone(for_test=True)

optimizer = optimizer_program()
optimizer.minimize(avg_cost)

exe = fluid.Executor(place)

EPOCH_NUM = 30

# For training test cost
def train_test(program, reader):
    count = 0
    feed_var_list = [
        program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder_test = fluid.DataFeeder(
        feed_list=feed_var_list, place=place)
    test_exe = fluid.Executor(place)
    accumulated = len([avg_cost, acc]) * [0]
    for tid, test_data in enumerate(reader()):
        avg_cost_np = test_exe.run(program=program,
                                   feed=feeder_test.feed(test_data),
                                   fetch_list=[avg_cost, acc])
        accumulated = [x[0] + x[1][0] for x in zip(accumulated, avg_cost_np)]
        count += 1
    return [x / count for x in accumulated]

In [8]:
params_dirname = "model/image_classification_vgg.inference.model"

from paddle.utils.plot import Ploter
import time

train_prompt = "Train cost"
test_prompt = "Test cost"
plot_cost = Ploter(test_prompt,train_prompt)
RESULT_FILE = 'result/results_vgg16.txt'

EPOCH_NUM = 30
print_every = 200
# main train loop.
def train_loop():
    feed_var_list_loop = [
        main_program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder = fluid.DataFeeder(
        feed_list=feed_var_list_loop, place=place)
    exe.run(star_program)

    
    
    start_time = time.time()
    for pass_id in range(EPOCH_NUM):
        average_loss = 0
        step = 1
        for step_id, data_train in enumerate(train_reader()):
            avg_loss_value = exe.run(main_program,
                                     feed=feeder.feed(data_train),
                                     fetch_list=[avg_cost, acc])
            average_loss += avg_loss_value[0]
            if step % print_every == 0:
#                 plot_cost.append(train_prompt, step, avg_loss_value[0])
#                 plot_cost.plot()
                average_loss = average_loss /print_every
                print('[epoch: %3d, step: %5d] loss: %.3f' % (pass_id + 1, step, average_loss))
                average_loss = 0
            step += 1
            
    total_time = time.time() - start_time
    with open(RESULT_FILE, 'a') as f:
        f.write('\n\n\ntraining results: \n')
        f.write('\n total training time: \t {}'.format(total_time))
        f.write('\n final average training lostt: \t %.3f' % (average_loss))

    start_time = time.time()
    avg_cost_test, accuracy_test = train_test(test_program, reader=test_reader)
    total_time = time.time() - start_time
#     plot_cost.append(test_prompt, step, avg_cost_test)
    s = '\ntesting results: \n accuracy on 1000 test images: %.3f, total time: %d s\n' % (accuracy_test, total_time)
    print(s)
    with open(RESULT_FILE, 'a') as f:
        f.write(s)

    # save parameters
#     if params_dirname is not None:
#         fluid.io.save_inference_model(params_dirname, ["pixel"],
#                                         [predict], exe)

In [9]:
train_loop()

[epoch:   1, step:   200] loss: 2.329
[epoch:   1, step:   400] loss: 2.109
[epoch:   1, step:   600] loss: 1.966
[epoch:   2, step:   200] loss: 1.849
[epoch:   2, step:   400] loss: 1.795
[epoch:   2, step:   600] loss: 1.743
[epoch:   3, step:   200] loss: 1.622
[epoch:   3, step:   400] loss: 1.565
[epoch:   3, step:   600] loss: 1.517
[epoch:   4, step:   200] loss: 1.463
[epoch:   4, step:   400] loss: 1.410
[epoch:   4, step:   600] loss: 1.380
[epoch:   5, step:   200] loss: 1.308
[epoch:   5, step:   400] loss: 1.268
[epoch:   5, step:   600] loss: 1.236
[epoch:   6, step:   200] loss: 1.192
[epoch:   6, step:   400] loss: 1.148
[epoch:   6, step:   600] loss: 1.137
[epoch:   7, step:   200] loss: 1.074
[epoch:   7, step:   400] loss: 1.079
[epoch:   7, step:   600] loss: 1.080
[epoch:   8, step:   200] loss: 1.009
[epoch:   8, step:   400] loss: 1.003
[epoch:   8, step:   600] loss: 0.986
[epoch:   9, step:   200] loss: 0.946
[epoch:   9, step:   400] loss: 0.963
[epoch:   9,

In [10]:
train_loop()

[epoch:   1, step:   200] loss: 2.325
[epoch:   1, step:   400] loss: 2.069
[epoch:   1, step:   600] loss: 1.969
[epoch:   2, step:   200] loss: 1.860
[epoch:   2, step:   400] loss: 1.790
[epoch:   2, step:   600] loss: 1.734
[epoch:   3, step:   200] loss: 1.653
[epoch:   3, step:   400] loss: 1.598
[epoch:   3, step:   600] loss: 1.561
[epoch:   4, step:   200] loss: 1.459
[epoch:   4, step:   400] loss: 1.417
[epoch:   4, step:   600] loss: 1.402
[epoch:   5, step:   200] loss: 1.302
[epoch:   5, step:   400] loss: 1.287
[epoch:   5, step:   600] loss: 1.254
[epoch:   6, step:   200] loss: 1.185
[epoch:   6, step:   400] loss: 1.150
[epoch:   6, step:   600] loss: 1.130
[epoch:   7, step:   200] loss: 1.074
[epoch:   7, step:   400] loss: 1.064
[epoch:   7, step:   600] loss: 1.055
[epoch:   8, step:   200] loss: 1.002
[epoch:   8, step:   400] loss: 0.997
[epoch:   8, step:   600] loss: 0.965
[epoch:   9, step:   200] loss: 0.939
[epoch:   9, step:   400] loss: 0.906
[epoch:   9,

In [11]:
train_loop()

[epoch:   1, step:   200] loss: 2.378
[epoch:   1, step:   400] loss: 2.130
[epoch:   1, step:   600] loss: 1.988
[epoch:   2, step:   200] loss: 1.857
[epoch:   2, step:   400] loss: 1.784
[epoch:   2, step:   600] loss: 1.730
[epoch:   3, step:   200] loss: 1.618
[epoch:   3, step:   400] loss: 1.561
[epoch:   3, step:   600] loss: 1.515
[epoch:   4, step:   200] loss: 1.432
[epoch:   4, step:   400] loss: 1.374
[epoch:   4, step:   600] loss: 1.349
[epoch:   5, step:   200] loss: 1.258
[epoch:   5, step:   400] loss: 1.237
[epoch:   5, step:   600] loss: 1.211
[epoch:   6, step:   200] loss: 1.149
[epoch:   6, step:   400] loss: 1.146
[epoch:   6, step:   600] loss: 1.127
[epoch:   7, step:   200] loss: 1.050
[epoch:   7, step:   400] loss: 1.067
[epoch:   7, step:   600] loss: 1.044
[epoch:   8, step:   200] loss: 1.002
[epoch:   8, step:   400] loss: 0.988
[epoch:   8, step:   600] loss: 0.964
[epoch:   9, step:   200] loss: 0.922
[epoch:   9, step:   400] loss: 0.932
[epoch:   9,